# Notebook práctico para el EFT: solución con LLM+RAG, agente funcional, observabilidad, trazabilidad y seguridad.

Este notebook demuestra la solución pedida en el EFT: carga de datos, RAG, agente funcional con memoria y planificación, observabilidad (precisión/latencia/consistencia), trazabilidad y protocolos de seguridad. Usa los módulos del repositorio (`agent/*`, `data/knowledge/*`, `storage/*`).

# 1. Importar librerías y configuración inicial
En VS Code y Colab: configurar entorno, cargar claves desde variables de entorno, definir rutas.



In [1]:
# Código: configuración mínima
import os, time, json, pathlib
from datetime import datetime

ROOT = pathlib.Path(r"c:/Users/Javier/Desktop/Instituto/IA/EV3/IAEva-main")
DATA_DIR = ROOT / "data" / "knowledge"
STORAGE_DIR = ROOT / "storage"
LOGS_FILE = STORAGE_DIR / "logs" / "interactions.jsonl"
NOTES_FILE = STORAGE_DIR / "notes.json"
CHROMA_DIR = STORAGE_DIR / "chroma"

os.environ.setdefault("LLM_API_KEY", os.getenv("LLM_API_KEY", "demo-key"))
print("Rutas:", DATA_DIR, STORAGE_DIR)


Rutas: c:\Users\Javier\Desktop\Instituto\IA\EV3\IAEva-main\data\knowledge c:\Users\Javier\Desktop\Instituto\IA\EV3\IAEva-main\storage


# 2. Cargar y preprocesar datos internos/externos


In [2]:
# Código: lectura simple de conocimiento interno
from pathlib import Path

files = list(DATA_DIR.glob("*.md"))
docs = []
for f in files:
    text = f.read_text(encoding="utf-8", errors="ignore")
    docs.append({"path": str(f), "text": text, "meta": {"source": f.name, "date": datetime.now().isoformat()}})

print(f"Documentos cargados: {len(docs)}")
print("Ejemplo fuente:", docs[0]["meta"]["source"] if docs else "sin documentos")


Documentos cargados: 2
Ejemplo fuente: politicas.md


# 3. Construir vector store y pipeline RAG


In [3]:
# Código: RAG mínimo con placeholders
# Nota: se usa un retriever simplificado (busca por palabras clave)

def simple_retriever(query, docs, k=3):
    q = query.lower().split()
    scored = []
    for d in docs:
        score = sum(word in d["text"].lower() for word in q)
        if score:
            scored.append((score, d))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [d for _, d in scored[:k]]

query = "protocolos de seguridad y uso responsable"
start = time.time()
retrieved = simple_retriever(query, docs, k=3)
latency_ms = (time.time() - start) * 1000
print(f"Chunks recuperados: {len(retrieved)} | Latencia: {latency_ms:.1f} ms")
for i, r in enumerate(retrieved, 1):
    print(i, r["meta"]["source"])

Chunks recuperados: 2 | Latencia: 0.1 ms
1 politicas.md
2 procedimientos.md


# 4. Ingeniería de prompts y plantillas


In [4]:
# Código: plantilla de prompt (JSON) para respuestas citadas
import textwrap

prompt_template = {
    "role": "analista",
    "task": "Responder con citas y políticas aplicables",
    "format": {
        "answer": "string",
        "citations": [{"source": "string", "snippet": "string"}],
        "confidence": "0-1"
    },
    "constraints": ["No inventar citas", "Respetar privacidad"],
}
print(json.dumps(prompt_template, indent=2, ensure_ascii=False))


{
  "role": "analista",
  "task": "Responder con citas y políticas aplicables",
  "format": {
    "answer": "string",
    "citations": [
      {
        "source": "string",
        "snippet": "string"
      }
    ],
    "confidence": "0-1"
  },
  "constraints": [
    "No inventar citas",
    "Respetar privacidad"
  ]
}


# 5. Orquestación del LLM con control de contexto


In [5]:
# Código: simulación de llamada LLM con contexto
# Se usa un "LLM" placeholder que combina plantilla + contexto recuperado.

def fake_llm_answer(query, retrieved, prompt):
    citations = [{"source": r["meta"]["source"], "snippet": r["text"][:120]} for r in retrieved]
    answer = f"Respuesta sobre: {query}. Basada en {len(retrieved)} fuentes."
    return {"answer": answer, "citations": citations, "confidence": 0.7}

resp = fake_llm_answer(query, retrieved, prompt_template)
print(json.dumps(resp, indent=2, ensure_ascii=False))


{
  "answer": "Respuesta sobre: protocolos de seguridad y uso responsable. Basada en 2 fuentes.",
  "citations": [
    {
      "source": "politicas.md",
      "snippet": "# Políticas de Comunicación Interna\n- Toda comunicación externa debe pasar por el área de Comunicaciones.\n- Los reportes"
    },
    {
      "source": "procedimientos.md",
      "snippet": "# Procedimiento de Onboarding\n1. Crear cuenta institucional.\n2. Asignar rutas de acceso.\n3. Revisión de políticas el pri"
    }
  ],
  "confidence": 0.7
}


# 6. Implementar agente con herramientas de consulta, escritura y razonamiento


In [6]:
# Código: agente mínimo con herramientas

def tool_write_note(note):
    notes = []
    if NOTES_FILE.exists():
        try:
            notes = json.loads(NOTES_FILE.read_text(encoding="utf-8"))
        except Exception:
            notes = []
    notes.append({"ts": datetime.now().isoformat(), "note": note})
    NOTES_FILE.write_text(json.dumps(notes, ensure_ascii=False, indent=2), encoding="utf-8")
    return True


def run_agent_task(query):
    start = time.time()
    retrieved_local = simple_retriever(query, docs, k=3)
    resp_local = fake_llm_answer(query, retrieved_local, prompt_template)
    tool_write_note(f"{resp_local['answer']} | fuentes: {[c['source'] for c in resp_local['citations']]}")
    duration = (time.time() - start) * 1000
    # log
    LOGS_FILE.parent.mkdir(parents=True, exist_ok=True)
    with open(LOGS_FILE, "a", encoding="utf-8") as f:
        f.write(json.dumps({
            "ts": datetime.now().isoformat(),
            "query": query,
            "latency_ms": duration,
            "sources": [c['source'] for c in resp_local['citations']]
        }, ensure_ascii=False) + "\n")
    return {"duration_ms": duration, "sources": [c['source'] for c in resp_local['citations']]}

metrics = run_agent_task("procedimientos y políticas de privacidad")
print("Ejecutado. Latencia:", f"{metrics['duration_ms']:.1f} ms", "| Fuentes:", metrics["sources"])

Ejecutado. Latencia: 0.7 ms | Fuentes: ['politicas.md', 'procedimientos.md']


# 7. Memoria del agente y recuperación de contexto


In [7]:
# Código: recuperación de notas previas (memoria persistente)
if NOTES_FILE.exists():
    notes = json.loads(NOTES_FILE.read_text(encoding="utf-8"))
    print(f"Notas almacenadas: {len(notes)}")
    print("Última nota:", notes[-1]["note"] if notes else "sin notas")
else:
    print("No hay notas aún.")


Notas almacenadas: 2
Última nota: Respuesta sobre: procedimientos y políticas de privacidad. Basada en 2 fuentes. | fuentes: ['politicas.md', 'procedimientos.md']


# 8. Planificación y toma de decisiones del agente


In [9]:
# Código: planner sencillo (descomposición de tarea)

def simple_planner(goal):
    return [
        {"step": 1, "action": "recuperar"},
        {"step": 2, "action": "razonar"},
        {"step": 3, "action": "escribir_nota"},
    ]

plan = simple_planner("Generar reporte de políticas de seguridad")
print(plan)


[{'step': 1, 'action': 'recuperar'}, {'step': 2, 'action': 'razonar'}, {'step': 3, 'action': 'escribir_nota'}]


# 9. Observabilidad: precisión, latencia y consistencia


In [10]:
# Código: métricas simples
import statistics

# precisión (proxy): coincidencia de palabras clave en respuesta
keywords = ["seguridad", "privacidad", "políticas"]
precision = sum(k in resp["answer"].lower() for k in keywords) / len(keywords)

# latencia: medimos de logs p95
latencies = []
if LOGS_FILE.exists():
    with open(LOGS_FILE, "r", encoding="utf-8") as f:
        for line in f:
            try:
                latencies.append(json.loads(line)["latency_ms"])
            except Exception:
                pass
p95 = statistics.quantiles(latencies, n=20)[18] if len(latencies) >= 20 else (max(latencies) if latencies else 0)

# consistencia: estabilidad entre runs (proxy: varianza de latencias)
variance = statistics.pvariance(latencies) if len(latencies) > 1 else 0

metrics_summary = {"precision": precision, "latency_p95_ms": round(p95,1), "latency_var": round(variance,1)}
print(metrics_summary)


{'precision': 0.3333333333333333, 'latency_p95_ms': 90208.7, 'latency_var': 819499290.5}


# 10. Trazabilidad: logging estructurado y análisis de registros


In [11]:
# Código: lectura y resumen de trazas
traces = []
if LOGS_FILE.exists():
    with open(LOGS_FILE, "r", encoding="utf-8") as f:
        for line in f:
            try:
                traces.append(json.loads(line))
            except Exception:
                pass
print(f"Registros: {len(traces)}")
print(traces[-1] if traces else "sin registros")


Registros: 8
{'ts': '2025-11-30T18:56:29.138179', 'query': 'procedimientos y políticas de privacidad', 'latency_ms': 0.5888938903808594, 'sources': ['politicas.md', 'procedimientos.md']}


# 11. Protocolos de seguridad, privacidad y uso responsable


In [12]:
# Código: validación simple de salida (no PII, no toxicidad)

def validate_output(resp):
    text = resp["answer"].lower()
    flags = {
        "pii": any(x in text for x in ["rut", "dni", "telefono", "email"]),
        "toxic": any(x in text for x in ["odio", "insulto"]),
    }
    return flags

print(validate_output(resp))


{'pii': False, 'toxic': False}


# 12. Propuestas de mejora basadas en métricas y registros


In [13]:
# Código: recomendaciones (placeholder)

def recommend_changes(metrics, traces):
    recs = []
    if metrics.get("latency_p95_ms", 0) > 500:
        recs.append("Agregar caché y reducir k en el retriever")
    if metrics.get("precision", 0) < 0.5:
        recs.append("Ajustar prompts y mejorar verificación de citas")
    recs.append("Instrumentar métricas adicionales en dashboard")
    return recs

print(recommend_changes(metrics_summary, traces))


['Agregar caché y reducir k en el retriever', 'Ajustar prompts y mejorar verificación de citas', 'Instrumentar métricas adicionales en dashboard']
